In [ ]:
import os
from typing import Tuple, Dict, Hashable, Iterable
import matplotlib.pyplot as plt
import h5py
import numpy as np
from numpy.typing import NDArray
import networkx as nx
import yaml

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 12,
})

def extend_coord(coord: NDArray, z: float) -> NDArray:
    return np.concatenate([coord, np.array([z])], axis=0)


def mid_coord(lhs: NDArray, rhs: NDArray) -> NDArray:
    return (lhs + rhs) / 2


def rot_and_proj(vec: NDArray, theta: float) -> NDArray:
    v2 = vec[0] * np.sin(theta) + vec[2] * np.cos(theta)
    return np.array([vec[1], v2])
     

def underlying_graph_layout(
    edges: Iterable[Tuple[int, int]],
) -> Dict[Hashable, NDArray]:
    graph = nx.Graph()
    for edge in edges:
        graph.add_edge(edge[0], edge[1])
    return nx.kamada_kawai_layout(graph)


def circuit_graph_and_layout_and_node_size(
    layers_number: int,
    nodes_number: int,
    edges: Iterable[Tuple[int, int]],
    theta: float,
) -> Tuple[nx.Graph, Dict[Hashable, NDArray], Dict[Hashable, float]]:
    slice_layout = underlying_graph_layout(edges)
    dz = 1. / (layers_number + 2)
    ddz = dz / len(edges + 1)
    graph = nx.Graph()
    front_layer_ids = {}
    layout = {}
    for i in range(nodes_number):
        new_id = "in", -1, i
        graph.add_node(new_id)
        front_layer_ids[i] = new_id
        coord = extend_coord(slice_layout[i], 0.)
        layout[new_id] = rot_and_proj(coord, theta)
    for layer_number in range(layers_number):
        for j, edge in enumerate(edges):
            new_id = "two", layer_number, edge[0], edge[1]
            graph.add_node(new_id)
            graph.add_edge(front_layer_ids[edge[0]], new_id)
            graph.add_edge(front_layer_ids[edge[1]], new_id)
            front_layer_ids[edge[0]] = new_id
            front_layer_ids[edge[1]] = new_id
            coord = extend_coord(mid_coord(slice_layout[edge[0]], slice_layout[edge[1]]), (layer_number + 1) * dz + j * ddz)
            layout[new_id] = rot_and_proj(coord, theta)
        for i in range(nodes_number):
            new_id = "one", layer_number, i
            graph.add_node(new_id)
            graph.add_edge(front_layer_ids[i], new_id)
            front_layer_ids[i] = new_id
            coord = extend_coord(slice_layout[i], (layer_number + 1) * dz + len(edges) * ddz)
            layout[new_id] = rot_and_proj(coord, theta)
    for i in range(nodes_number):
        new_id = "out", layers_number, i
        graph.add_node(new_id)
        graph.add_edge(front_layer_ids[i], new_id)
        coord = extend_coord(slice_layout[i], 1)
        layout[new_id] = rot_and_proj(coord, theta)
    node_size = [30. if v[0] != "out" and v[0] != "in" else 0. for v in graph]
    return graph, layout, node_size

script_dir = os.getcwd()
# ---------- here is the path to the results directory ----------------------
results_dir_path = f"{script_dir}/outputs/qbp/very_large/2024-05-18/11-00-16"
# ---------------------------------------------------------------------------
results_path = f"{results_dir_path}/result"
config_path = f"{results_dir_path}/.hydra/config.yaml"
with open(config_path, "r") as f:
    raw_config = f.read()
    config = yaml.safe_load(raw_config)
layers_number = 10
with h5py.File(results_path, "r") as f:
    fields = np.array(f["fields"])
    pairs = np.array(f["coupled_spin_pairs"])
    tn_graph, layout, node_size = circuit_graph_and_layout_and_node_size(layers_number, fields.shape[0], pairs, 0.0)
    fig = plt.figure(figsize=(15, 30))
    nx.draw(tn_graph, pos=layout, node_size=node_size, width=0.75, edge_color='black')

In [42]:
fig.savefig(f"{results_dir_path}/tensor_network.pdf", bbox_inches="tight")